In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.datasets.qm9 import QM9
import torch_geometric.datasets.qm9 as qm9
from torch_geometric.data import DataLoader
import torch_geometric.nn as tgnn
from torch_scatter import scatter
import tqdm
import numpy as np
import wandb

In [2]:
# [0] Reports MAE in eV / Chemical Accuracy of the target variable U0. 
# The chemical accuracy of U0 is 0.043 see [1, Table 5].

# Reproduced table [0]
# MXMNet: 0.00590/0.043 = 0.13720930232558143
# HMGNN:  0.00592/0.043 = 0.13767441860465118
# MPNN:   0.01935/0.043 = 0.45
# KRR:    0.0251 /0.043 = 0.5837209302325582
# [0] https://paperswithcode.com/sota/formation-energy-on-qm9
# [1] Neural Message Passing for Quantum Chemistry, https://arxiv.org/pdf/1704.01212v2.pdf
# MXMNet https://arxiv.org/pdf/2011.07457v1.pdf
# HMGNN https://arxiv.org/pdf/2009.12710v1.pdf
# MPNN https://arxiv.org/pdf/1704.01212v2.pdf
# KRR HDAD kernel ridge regression https://arxiv.org/pdf/1702.05532.pdf
# HDAD means HDAD (Histogram of distances, anglesand dihedral angles)

# [2] Reports the average value of MAE / Chemical Accuracy of over all targets
# [2] https://paperswithcode.com/sota/drug-discovery-on-qm9
target_dict = {0: 'mu, D, Dipole moment', 
               1: 'alpha, {a_0}^3, Isotropic polarizability', 
               2: 'epsilon_{HOMO}, eV, Highest occupied molecular orbital energy',
               3: 'epsilon_{LUMO}, eV, Lowest unoccupied molecular orbital energy',
               4: 'Delta, eV, Gap between HOMO and LUMO',
               5: '< R^2 >, {a_0}^2, Electronic spatial extent',
               6: 'ZPVE, eV, Zero point vibrational energy', 
               7: 'U_0, eV, Internal energy at 0K',
               8: 'U, eV, Internal energy at 298.15K', 
               9: 'H, eV, Enthalpy at 298.15K',
               10: 'G, eV, Free energy at 298.15K',  
               11: 'c_{v}, cal\(mol K), Heat capacity at 298.15K'}

chemical_accuracy = {idx:0.043 for idx in range(12)}
chemical_accuracy[0] = 0.1
chemical_accuracy[1] = 0.1
chemical_accuracy[5] = 1.2
chemical_accuracy[6] = 0.0012
chemical_accuracy[11] = 0.050

In [3]:
wandb.init(project='QM9', entity='chrisxx')
config = wandb.config
config.lr = 0.0003
config.n_epochs = 10000
config.patience = 5
config.factor = 0.9
config.target_idx = 7
config.batch_size = 128
config.n_train = 110000
config.n_valid = 10000

wandb: Currently logged in as: chrisxx (use `wandb login --relogin` to force relogin)


In [4]:
target_idx = config.target_idx

In [5]:
dataset = QM9('../datasets/qm9_geometric/')

In [6]:
dataset = dataset.shuffle()
train_dataset = dataset[:config.n_train]
valid_dataset = dataset[config.n_train:config.n_train+config.n_valid]
test_dataset = dataset[config.n_train+config.n_valid:]
config.n_test = len(test_dataset)
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config.batch_size)
test_loader = DataLoader(test_dataset, batch_size=8)

In [7]:
ngpu=1
device = torch.device('cuda:0' if (torch.cuda.is_available() and ngpu > 0) else 'cpu')

# Model

In [8]:
target_vec = []

In [9]:
# based on https://schnetpack.readthedocs.io/en/stable/tutorials/tutorial_02_qm9.html
# and https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/nn/models/schnet.html#SchNet
for data in train_loader:
    data = data.to(device)
    atomU0s = torch.tensor(qm9.atomrefs[target_idx], device=device)[torch.argmax(data.x[:, :5], axis=1)]
    target_modular = scatter(atomU0s, data.batch, dim=-1, reduce='sum')
    target_vec += [(data.y[:, target_idx] - target_modular).detach().cpu().numpy()]
target_vec = np.concatenate(target_vec, axis=0)

In [10]:
target_mean = np.mean(target_vec)
target_std = np.std(target_vec)

In [11]:
model = tgnn.SchNet(hidden_channels=128, num_filters=128, num_interactions=6, num_gaussians=50, cutoff=10.0,
                    readout='add', atomref=dataset.atomref(target_idx), mean=target_mean, std=target_std)
loss = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=config.lr)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', verbose=True, patience=config.patience, factor=config.factor)

# Training

In [12]:
model = model.to(device)

In [13]:
loss_hist = []
try:
    # For each epoch
    for epoch in range(config.n_epochs):
        # For each batch in the dataloader
        pbar = tqdm.tqdm(enumerate(train_loader, 0))
        epoch_loss = 0
        for i, data in pbar:
            model.zero_grad()
            data.to(device)
            target = data.y[:, target_idx]
            prediction = model(data.z, data.pos, data.batch)
            output = loss(prediction.view(-1), target)
            mae = (prediction.view(-1) - target).abs().mean()
            epoch_loss = (epoch_loss*i + mae.item())/(i+1)
            
            pbar.set_description('Epoch %d: MAE/CA %2.6f'%(epoch+1, epoch_loss/chemical_accuracy[target_idx]))
            output.backward()
            optimizer.step()
            wandb.log({'MSE': output.item()})
        wandb.log({'MAE':epoch_loss, 
                   'MAE/CA':epoch_loss/chemical_accuracy[target_idx]})
        lr_scheduler.step(epoch_loss)
        loss_hist += [epoch_loss]
        if epoch_loss/chemical_accuracy[target_idx] < 0.45:
            break

except KeyboardInterrupt:
    print('keyboard interrupt caught')

Epoch 1: MAE/CA 427.416600: : 860it [00:28, 29.75it/s] 
Epoch 2: MAE/CA 107.439756: : 860it [00:27, 31.02it/s]
Epoch 3: MAE/CA 98.632933: : 860it [00:27, 31.24it/s] 
Epoch 4: MAE/CA 85.642215: : 860it [00:27, 31.19it/s] 
Epoch 5: MAE/CA 78.977036: : 860it [00:27, 31.20it/s]
Epoch 6: MAE/CA 65.881465: : 860it [00:27, 31.21it/s] 
Epoch 7: MAE/CA 60.796242: : 860it [00:27, 31.21it/s]
Epoch 8: MAE/CA 66.145693: : 860it [00:27, 31.22it/s]
Epoch 9: MAE/CA 63.079950: : 860it [00:27, 31.26it/s]
Epoch 10: MAE/CA 65.966018: : 860it [00:27, 31.23it/s] 
Epoch 11: MAE/CA 52.733130: : 860it [00:27, 31.02it/s]
Epoch 12: MAE/CA 57.578002: : 860it [00:27, 31.19it/s]
Epoch 13: MAE/CA 50.951289: : 860it [00:27, 31.13it/s]
Epoch 14: MAE/CA 43.412078: : 860it [00:27, 31.28it/s]
Epoch 15: MAE/CA 50.793904: : 860it [00:27, 31.30it/s]
Epoch 16: MAE/CA 48.653033: : 860it [00:27, 31.26it/s]
Epoch 17: MAE/CA 32.620593: : 860it [00:27, 31.27it/s]
Epoch 18: MAE/CA 38.462694: : 860it [00:27, 31.23it/s]
Epoch 19: MA

Epoch    48: reducing learning rate of group 0 to 2.7000e-04.


Epoch 49: MAE/CA 5.777294: : 860it [00:27, 31.00it/s] 
Epoch 50: MAE/CA 13.928737: : 860it [00:28, 30.04it/s]
Epoch 51: MAE/CA 12.347787: : 860it [00:29, 29.20it/s]
Epoch 52: MAE/CA 13.074337: : 860it [00:28, 29.67it/s]
Epoch 53: MAE/CA 12.815140: : 860it [00:28, 29.67it/s]
Epoch 54: MAE/CA 10.394314: : 860it [00:29, 29.21it/s]


Epoch    54: reducing learning rate of group 0 to 2.4300e-04.


Epoch 55: MAE/CA 9.020215: : 860it [00:29, 29.32it/s]
Epoch 56: MAE/CA 10.938326: : 860it [00:29, 29.27it/s]
Epoch 57: MAE/CA 11.268693: : 860it [00:29, 29.02it/s]
Epoch 58: MAE/CA 10.887005: : 860it [00:29, 28.83it/s]
Epoch 59: MAE/CA 10.090998: : 860it [00:29, 29.05it/s]
Epoch 60: MAE/CA 10.737415: : 860it [00:29, 29.15it/s]


...

Epoch   500: reducing learning rate of group 0 to 1.9088e-06.


Epoch 501: MAE/CA 0.768120: : 860it [00:27, 30.96it/s]
Epoch 502: MAE/CA 0.763784: : 860it [00:27, 30.96it/s]
Epoch 503: MAE/CA 0.768837: : 860it [00:27, 30.85it/s]
Epoch 504: MAE/CA 0.763769: : 860it [00:27, 30.94it/s]
Epoch 505: MAE/CA 0.772736: : 860it [00:27, 30.93it/s]
Epoch 506: MAE/CA 0.770236: : 860it [00:27, 31.03it/s]
Epoch 507: MAE/CA 0.769637: : 488it [00:15, 30.85it/s]


Exception: The wandb backend process has shutdown

Error in callback <function _WandbInit._pause_backend at 0x000002A610DD95E0> (for post_run_cell):


Exception: The wandb backend process has shutdown

In [ ]:
model.to('cpu')

In [ ]:
pbar = tqdm.tqdm(enumerate(test_loader, 0))
epoch_loss = 0
maes = []
for i, data in pbar:
    data.to('cpu')
    prediction = model(data.z, data.pos, data.batch)
    mae = (prediction.view(-1) - data.y[:, target_idx]).abs()
    maes += [mae]
maes = torch.cat(maes, dim=0)
mae = maes.mean().item()
print(mae, mae/chemical_accuracy[target_idx])
wandb.log({'TEST MAE':mae, 'TEST MAE/CA':mae/chemical_accuracy[target_idx]})


Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Program Files\Python38\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Python38\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "c:\users\chris\appdata\local\pypoetry\cache\virtualenvs\graph-transformer-tx-dblaw-py3.8\lib\site-packages\wandb\sdk\wandb_run.py", line 201, in check_status
    status_response = self._interface.communicate_stop_status()
  File "c:\users\chris\appdata\local\pypoetry\cache\virtualenvs\graph-transformer-tx-dblaw-py3.8\lib\site-packages\wandb\sdk\interface\interface.py", line 743, in communicate_stop_status
    resp = self._communicate(req, timeout=timeout, local=True)
  File "c:\users\chris\appdata\local\pypoetry\cache\virtualenvs\graph-transformer-tx-dblaw-py3.8\lib\site-packages\wandb\sdk\interface\interface.py", line 545, in _communicate
    return self._communicate_async(rec, local=local).get(timeout=